In [20]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import pickle
import matplotlib.patches as patches
import re
import random 
from sklearn.model_selection import train_test_split
import cv2
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
from PIL import Image
import tensorflow as tf
from keras.layers import GlobalAveragePooling2D,Dense,Conv2D,BatchNormalization,Dropout
import keras
from keras import backend as K
from keras.models import Model,load_model
from tensorflow.python.framework.ops import disable_eager_execution
from keras.regularizers import l2
import datetime

In [2]:
train_csv=pd.read_csv('archive/train.csv')
train_csv.head()
     

,ImageId,ClassId,EncodedPixels
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 388...
3,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 ...
4,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468...


In [3]:
Image_id=[]
label=[]
train_folder_path='archive/train_images'
for i in os.listdir(train_folder_path): #https://www.geeksforgeeks.org/python-os-listdir-method/
    for j in range(1,5):
        Image_id.append(i)
        label.append(j)

x={'ImageId':Image_id,'ClassId':label}  #https://www.geeksforgeeks.org/creating-a-pandas-dataframe/
train_img=pd.DataFrame(x)
train_img.head(10)

,ImageId,ClassId
0,0002cc93b.jpg,1
1,0002cc93b.jpg,2
2,0002cc93b.jpg,3
3,0002cc93b.jpg,4
4,00031f466.jpg,1
5,00031f466.jpg,2
6,00031f466.jpg,3
7,00031f466.jpg,4
8,000418bfc.jpg,1
9,000418bfc.jpg,2


In [4]:
df=pd.merge(train_img,train_csv,how='outer',on=['ImageId','ClassId'])
df.fillna('',inplace=True)
df.head()

,ImageId,ClassId,EncodedPixels
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0002cc93b.jpg,2,
2,0002cc93b.jpg,3,
3,0002cc93b.jpg,4,
4,00031f466.jpg,1,


In [5]:
#https://www.analyticsvidhya.com/blog/2020/03/pivot-table-pandas-python/
train=pd.pivot_table(df,values='EncodedPixels',index='ImageId',columns='ClassId',aggfunc=np.sum).astype(str)
train=train.reset_index()
train.columns=['image_id','rle_1','rle_2','rle_3','rle_4'] 
train.head()

,image_id,rle_1,rle_2,rle_3,rle_4
0,0002cc93b.jpg,29102 12 29346 24 29602 24 29858 24 30114 24 3...,,,
1,00031f466.jpg,,,,
2,000418bfc.jpg,,,,
3,000789191.jpg,,,,
4,0007a71bf.jpg,,,18661 28 18863 82 19091 110 19347 110 19603 11...,


In [6]:
defect=[]
stratify=[]
for i in range(len(train)):
    if (train['rle_1'][i] != '' or train['rle_2'][i] != '' or train['rle_3'][i] != '' or train['rle_4'][i] != ''):
         defect.append(1)
    else:
         defect.append(0)
  
    if train['rle_1'][i] != '':
         stratify.append(1)
    elif train['rle_2'][i] != '':
        stratify.append(2)
    elif train['rle_3'][i] != '':
        stratify.append(3)
    elif train['rle_4'][i] != '':
        stratify.append(4)
    else:
        stratify.append(0)
train['defect']=defect
train['stratify']=stratify

In [7]:
train.head()

,image_id,rle_1,rle_2,rle_3,rle_4,defect,stratify
0,0002cc93b.jpg,29102 12 29346 24 29602 24 29858 24 30114 24 3...,,,,1,1
1,00031f466.jpg,,,,,0,0
2,000418bfc.jpg,,,,,0,0
3,000789191.jpg,,,,,0,0
4,0007a71bf.jpg,,,18661 28 18863 82 19091 110 19347 110 19603 11...,,1,3


In [8]:
defect_1,defect_2,defect_3,defect_4=[],[],[],[]
for i in range(len(train)):
    if train['rle_1'][i] != '':
         defect_1.append(1)
    else:
         defect_1.append(0)
    if train['rle_2'][i] != '':
         defect_2.append(1)
    else:
         defect_2.append(0)
    if train['rle_3'][i] != '':
         defect_3.append(1)
    else:
         defect_3.append(0)
    if train['rle_4'][i] != '':
        defect_4.append(1)
    else:
        defect_4.append(0)
train['defect_1']=defect_1
train['defect_2']=defect_2
train['defect_3']=defect_3
train['defect_4']=defect_4
train['total_defects']=train['defect_1']+ train['defect_2']+ train['defect_3']+ train['defect_4']
train.head()

,image_id,rle_1,rle_2,rle_3,rle_4,defect,stratify,defect_1,defect_2,defect_3,defect_4,total_defects
0,0002cc93b.jpg,29102 12 29346 24 29602 24 29858 24 30114 24 3...,,,,1,1,1,0,0,0,1
1,00031f466.jpg,,,,,0,0,0,0,0,0,0
2,000418bfc.jpg,,,,,0,0,0,0,0,0,0
3,000789191.jpg,,,,,0,0,0,0,0,0,0
4,0007a71bf.jpg,,,18661 28 18863 82 19091 110 19347 110 19603 11...,,1,3,0,0,1,0,1


In [9]:
train.shape

(12568, 12)

In [10]:
import tensorflow as tf

def f1_score(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(tf.greater(y_pred, 0.5), tf.float32)  # Threshold predictions

    true_positives = tf.reduce_sum(y_true * y_pred, axis=0)
    possible_positives = tf.reduce_sum(y_true, axis=0)
    predicted_positives = tf.reduce_sum(y_pred, axis=0)

    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())

    f1 = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
    return tf.reduce_mean(f1)



In [11]:
train=train[train['defect']==1]
x_train,x_test=train_test_split(train,test_size=0.10,stratify=train['stratify'],random_state=0)
x_train,x_val=train_test_split(x_train,test_size=0.20,stratify=x_train['stratify'],random_state=0)
x_train=x_train[['image_id','defect_1','defect_2','defect_3','defect_4']]
x_val=x_val[['image_id','defect_1','defect_2','defect_3','defect_4']]
x_test=x_test[['image_id','defect_1','defect_2','defect_3','defect_4']]
print("x_train {}".format(x_train.shape),"  x_val {}".format(x_val.shape)," x_test {}".format(x_test.shape))

x_train (4799, 5)   x_val (1200, 5)  x_test (667, 5)


In [12]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.2,
                                 zoom_range=0.1,
                                 horizontal_flip=True,
                                 vertical_flip=True,
                                 rotation_range=10,
                                 width_shift_range=0.1, 
                                 height_shift_range=0.1)

val_datagen=ImageDataGenerator(rescale=1./255)

train_folder_path='archive/train_images'
col=["defect_1","defect_2","defect_3","defect_4"]
train_image_generator=train_datagen.flow_from_dataframe(dataframe=x_train,
                                                  directory=train_folder_path,
                                                  x_col="image_id",
                                                  y_col=col,
                                                  batch_size=32,
                                                  class_mode="raw",
                                                  target_size=(256,512))

val_image_generator=val_datagen.flow_from_dataframe(dataframe=x_val,
                                                  directory=train_folder_path,
                                                  x_col="image_id",
                                                  y_col=col,
                                                  batch_size=32,
                                                  class_mode="raw",
                                                  target_size=(256,512))

Found 4799 validated image filenames.
Found 1200 validated image filenames.


In [13]:
#https://keras.io/api/applications/inceptionresnetv2/
base_model=tf.keras.applications.xception.Xception(input_shape=(256,512,3),include_top=False)
base_model.trainable=False

m=base_model.output
# add a global average pooling layer
#https://stackoverflow.com/questions/49295311/what-is-the-difference-between-flatten-and-globalaveragepooling2d-in-keras 
m=GlobalAveragePooling2D()(m)

# add fully-connected layers
m=Dense(1024,activation='relu')(m)
m=BatchNormalization()(m)   #https://www.analyticsvidhya.com/blog/2021/03/introduction-to-batch-normalization/
m=Dropout(0.4)(m)

#https://machinelearningmastery.com/how-to-reduce-overfitting-in-deep-learning-with-weight-regularization/
m=Dense(512,activation='relu')(m) 
m=BatchNormalization()(m)
m= Dropout(0.4)(m)

m=Dense(64,activation='relu')(m)

#prediction layer
output=Dense(4,activation='sigmoid')(m)   

model=Model(inputs=base_model.input,outputs=output)
model._name="Multi_label_Classification_Model"
model.summary()
     

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 256, 512, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv1 (Conv2D)         │ (None, 127, 255, 32)      │             864 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv1_bn               │ (None, 127, 255, 32)      │             128 │ block1_conv1[0][0]         │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv1_act (Activation) │ (None, 127, 255, 32)      │               0 │ block1_conv1_bn[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv2 (Conv2D)         │ (None, 125, 253, 64)      │          18,432 │ block1_conv1_act[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv2_bn               │ (None, 125, 253, 64)      │             256 │ block1_conv2[0][0]         │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv2_act (Activation) │ (None, 125, 253, 64)      │               0 │ block1_conv2_bn[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_sepconv1               │ (None, 125, 253, 128)     │           8,768 │ block1_conv2_act[0][0]     │
│ (SeparableConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_sepconv1_bn            │ (None, 125, 253, 128)     │             512 │ block2_sepconv1[0][0]      │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_sepconv2_act           │ (None, 125, 253, 128)     │               0 │ block2_sepconv1_bn[0][0]   │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_sepconv2               │ (None, 125, 253, 128)     │          17,536 │ block2_sepconv2_act[0][0]  │
│ (SeparableConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_sepconv2_bn            │ (None, 125, 253, 128)     │             512 │ block2_sepconv2[0][0]      │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 63, 127, 128)      │           8,192 │ block1_conv2_act[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_pool (MaxPooling2D)    │ (None, 63, 127, 128)      │               

 Total params: 23,523,692 (89.74 MB)

 Trainable params: 2,659,140 (10.14 MB)

 Non-trainable params: 20,864,552 (79.59 MB)

In [14]:
import os
import datetime
import tensorflow as tf

# Define the path to save the model
checkpoint_filepath = 'archive/multi_label.keras'  # Ensure the correct path and extension

# Create TensorBoard callback
log_dir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True)

# Define the ModelCheckpoint callback
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_f1_score',  # Monitor validation F1 score
    mode='max',             # Mode for saving the best model
    save_best_only=True     # Save only the best model
)

# Compile the model with f1_score as a metric
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=["acc", f1_score])

# Define callbacks including TensorBoard and ModelCheckpoint
callbacks = [model_checkpoint_callback, tensorboard]

# Train the model
history = model.fit(
    train_image_generator,
    validation_data=val_image_generator,
    epochs=20,
    verbose=1,
    callbacks=callbacks
)



Epoch 1/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 1559s 10s/step - acc: 0.6888 - f1_score: 0.3955 - loss: 0.4038 - val_acc: 0.8008 - val_f1_score: 0.5079 - val_loss: 0.2507
Epoch 2/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 1283s 8s/step - acc: 0.7896 - f1_score: 0.5132 - loss: 0.2567 - val_acc: 0.8392 - val_f1_score: 0.6069 - val_loss: 0.2059
Epoch 3/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 1238s 8s/step - acc: 0.8177 - f1_score: 0.5724 - loss: 0.2270 - val_acc: 0.8408 - val_f1_score: 0.6735 - val_loss: 0.1940
Epoch 4/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 1412s 9s/step - acc: 0.8142 - f1_score: 0.5786 - loss: 0.2263 - val_acc: 0.8500 - val_f1_score: 0.6281 - val_loss: 0.1817
Epoch 5/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 29041s 195s/step - acc: 0.8323 - f1_score: 0.5882 - loss: 0.1993 - val_acc: 0.8442 - val_f1_score: 0.7163 - val_loss: 0.1717
Epoch 6/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 1205s 8s/step - acc: 0.8248 - f1_score: 0.6384 - loss: 0.2063 - val_acc: 0.8383 - val_f1_score: 0.7002 - val_loss: 0.1800
Epoch 7/20
150/150 ━━━━━

In [22]:
%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 66696), started 7 days, 1:36:41 ago. (Use '!kill 66696' to kill it.)

In [16]:
train_image_generator=val_datagen.flow_from_dataframe(dataframe=x_train,
                                                  directory=train_folder_path,
                                                  x_col="image_id",
                                                  y_col=col,
                                                  batch_size=32,
                                                  shuffle=False,
                                                  class_mode="raw",
                                                  target_size=(256,512))


val_image_generator=val_datagen.flow_from_dataframe(dataframe=x_val,
                                                  directory=train_folder_path,
                                                  x_col="image_id",
                                                  y_col=col,
                                                  batch_size=32,
                                                  shuffle=False,
                                                  class_mode="raw",
                                                  target_size=(256,512))

test_image_generator=val_datagen.flow_from_dataframe(dataframe=x_test,
                                                  directory=train_folder_path,
                                                  x_col="image_id",
                                                  y_col=col,
                                                  batch_size=32,
                                                  shuffle=False,
                                                  class_mode="raw",
                                                  target_size=(256,512))

Found 4799 validated image filenames.
Found 1200 validated image filenames.
Found 667 validated image filenames.


In [17]:
print('Training Dataset:\n')
print(model.evaluate(train_image_generator,verbose=1))
print("="*100)
print('\nValidation Dataset:\n')
print(model.evaluate(val_image_generator,verbose=1))
print("="*100)
print('\nTest Dataset:\n')
print(model.evaluate(test_image_generator,verbose=1))

Training Dataset:

150/150 ━━━━━━━━━━━━━━━━━━━━ 929s 6s/step - acc: 0.8806 - f1_score: 0.7899 - loss: 0.1284
[0.12493725121021271, 0.8845592737197876, 0.7804034948348999]

Validation Dataset:

38/38 ━━━━━━━━━━━━━━━━━━━━ 235s 6s/step - acc: 0.8503 - f1_score: 0.7712 - loss: 0.1540
[0.16566826403141022, 0.8533333539962769, 0.7454371452331543]

Test Dataset:

21/21 ━━━━━━━━━━━━━━━━━━━━ 131s 6s/step - acc: 0.8591 - f1_score: 0.6842 - loss: 0.1622
[0.15476520359516144, 0.8545727133750916, 0.7111184000968933]
